In [35]:
import requests
import pandas as pd
import hvplot.pandas
import json
import holoviews as hv

api_key = "a9b5c9761f814b12b3b59d73ca004a4d"

In [4]:
data = pd.read_csv("theme_park_injuries.csv")



#Halloween Horror Nights - drop, not a ride
#Studio Backlot Tour - drop 
#Honey I Shrunk The Kids - demolished to make room for Star Wars: Galaxy's Edge
drop_areas = data[(data["Ride_name"] != "Halloween Horror Nights") &
                 (data["Ride_name"] != "Studio Backlot Tour") &
                 (data["Ride_name"] != "Honey I Shrunk The Kids")].copy()

#correct company name
drop_areas.loc[drop_areas["Company"] == "Universal:", "Company"] = "Universal"

#further cleaning data - some rides have been redesigned/renamed
#Jurassic World VelociCoaster -> VelociCoaster
#Miss Adventure Falls -> Typhoon Tillie's
#Oh! No! Drop Slide -> Ohno Drop Slide
#Terminator 2: 3D -> The Bourne Stuntacular

drop_areas.loc[drop_areas["Ride_name"] == "Dinosaur", "Ride_name"] = "The Dino Institute Shop"
drop_areas.loc[drop_areas["Ride_name"] == "Disaster", "Ride_name"] = "Fast & Furious - Supercharged"
drop_areas.loc[drop_areas["Ride_name"] == "Gran Fiesta Tour Attraction", "Ride_name"] = "Plaza de Los Amigos"
drop_areas.loc[drop_areas["Ride_name"] == "Jurassic World VelociCoaster", "Ride_name"] = "VelociCoaster"
drop_areas.loc[drop_areas["Ride_name"] == "Oh! No! Drop Slide", "Ride_name"] = "Ohno Drop Slide"
drop_areas.loc[drop_areas["Ride_name"] == "Miss Adventure Falls", "Ride_name"] = "Typhoon Tillie's"
drop_areas.loc[drop_areas["Ride_name"] == "Terminator 2: 3D", "Ride_name"] = "The Bourne Stuntacular"
drop_areas.loc[drop_areas["Ride_name"] == "Dragon Challenge", "Ride_name"] = "Hagrid's Magical Creatures Motorbike Adventure"
drop_areas.loc[drop_areas["Ride_name"] == "The Liberty Belle", "Ride_name"] = "Liberty Square Riverboat Landing"
drop_areas.loc[drop_areas["Ride_name"] == "Kala Serpentine Body Slide", "Ride_name"] = "Kala & Tai Nui Serpentine Body Slides"
drop_areas.loc[drop_areas["Ride_name"] == "Tai Nui Serpentine Body Slide", "Ride_name"] = "Kala & Tai Nui Serpentine Body Slides"
drop_areas.loc[drop_areas["Ride_name"] == "Chair Lift", "Ride_name"] = "Arctic Expeditions"
drop_areas.loc[drop_areas["Ride_name"] == "Meltaway Bay", "Ride_name"] = "Melt-away Bay"
drop_areas.loc[drop_areas["Ride_name"] == "Snow White's Scary Adventures", "Ride_name"] = "Princess Fairytale Hall"
drop_areas.loc[drop_areas["Ride_name"] == "Primeval Whirl", "Ride_name"] = "Dino Diner"
drop_areas.loc[drop_areas["Ride_name"] == "Ellen's Energy Adventure", "Ride_name"] = "Guardians of the Galaxy: Cosmic Rewind"
drop_areas.loc[drop_areas["Ride_name"] == "Dr Doom's Fearfall", "Ride_name"] = "Cafe 4"
drop_areas.loc[drop_areas["Ride_name"] == "Summit Plummet", "Ride_name"] = "Mount Gushmore Station"
drop_areas.loc[drop_areas["Ride_name"] == "Storm Slide-Rudder Buster", "Ride_name"] = "Storm Slides"
drop_areas.loc[drop_areas["Ride_name"] == "Harry Potter and the Forbidden Journey", "Theme_Park"] = "Islands of Adventure"
drop_areas.loc[drop_areas["Ride_name"] == "Castaway Creek", "Theme_Park"] = "Typhoon Lagoon"
drop_areas.loc[drop_areas["Ride_name"] == "Crush n' Gusher", "Theme_Park"] = "Typhoon Lagoon"
drop_areas.loc[drop_areas["Ride_name"] == "Slush Gusher", "Theme_Park"] = "Typhoon Lagoon"
drop_areas.loc[(drop_areas["Ride_name_dirty"] == "Transformers: The Ride 3D"), ["Company"]] = ["Disney"]
drop_areas.loc[(drop_areas["Ride_name_dirty"] == "Blizzard Beach") & (drop_areas["Ride_name"] == "Surf Pool"), "Ride_name"] = "Melt-away Bay"

#create list of unique ride name to use for retrieving coordinates
rides = pd.DataFrame(drop_areas["Ride_name"].unique(), columns=["Ride_name"])

rides.head()

drop_areas.head()

,Company,Incident_date,Ride_name_dirty,Ride_name,Theme_Park,age_gender,description
0,Disney World,6/9/2022,Alien Swirling Saucers,Alien Swirling Saucers,Hollywood Studios,59 yof,guest with pre-existing condition sustained in...
1,Disney World,10/5/2014,Astro Orbiter,Astro Orbiter,Magic Kingdom,68 yom,"guest fell exiting vehicle, developed blood c..."
2,Disney World,12/3/2009,Astro Orbitor,Astro Orbiter,Magic Kingdom,40 yof,fractured left ankle exiting ride
3,Disney World,8/7/2022,Avatar Flight of Passage,Avatar Flight of Passage,Animal Kingdom,83 yom,guest with pre-existing condition was briefly ...
4,Disney World,4/19/2022,Avatar Flight of Passage,Avatar Flight of Passage,Animal Kingdom,42 yof,guest with pre-existing condition had motion s...


In [5]:
orl_lat = 28.5384
orl_lon = -81.3789
radius = 75000

params = {
    "apiKey":api_key,
    "filter":f"circle:{orl_lon},{orl_lat},{radius}"
}

#add empty columns for lat, lon
rides["lat"] = None
rides["lon"] = None

    
base_url = "https://api.geoapify.com/v1/geocode/search"

for index, row in rides.iterrows():
    ride_name = row["Ride_name"]
    params["text"] = ride_name
    ride_data = requests.get(base_url, params=params).json()
    try:
        rides.loc[index, "lat"] = ride_data["features"][0]["properties"]["lat"]
        rides.loc[index, "lon"] = ride_data["features"][0]["properties"]["lon"]
    except IndexError:
        print("No record found")
        
rides.head()


,Ride_name,lat,lon
0,Alien Swirling Saucers,28.355652,-81.562734
1,Astro Orbiter,28.418427,-81.579153
2,Avatar Flight of Passage,28.354861,-81.592826
3,Back to the Future,28.480506,-81.467064
4,Big Thunder Mountain Railroad,28.420193,-81.58441


In [6]:
#merge incident count with ride name location
incident_count = drop_areas.groupby(["Company", "Theme_Park", "Ride_name"]).size().rename("incident_count").reset_index()
incident_count.head()

mapping_data = pd.merge(rides, incident_count, how="inner", on="Ride_name")
mapping_data.head()




,Ride_name,lat,lon,Company,Theme_Park,incident_count
0,Alien Swirling Saucers,28.355652,-81.562734,Disney World,Hollywood Studios,1
1,Astro Orbiter,28.418427,-81.579153,Disney World,Magic Kingdom,2
2,Avatar Flight of Passage,28.354861,-81.592826,Disney World,Animal Kingdom,15
3,Back to the Future,28.480506,-81.467064,Universal,Universal Studios,1
4,Big Thunder Mountain Railroad,28.420193,-81.58441,Disney World,Magic Kingdom,12


In [84]:
def plot_mapping_data(mapping_data, filter_column=None, 
                      filter_value=None, scale=5, save_file=None, title="title"):
    
    if filter_column is not None and filter_value is not None:
        filtered_data = mapping_data[mapping_data[filter_column] == filter_value]
    else:
        filtered_data = mapping_data
    
    map_plot = filtered_data.hvplot.points("lon",
                                     "lat",
                                     legend=True,
                                     color="Ride_name",
                                     tiles="CartoLight",
                                     size="incident_count",
                                     scale=scale,
                                     width=1000,
                                     height=750,
                                     alpha=0.75,
                                     title=title)
    
    hvplot.save(map_plot, f"maps\{title}.png")
    
    return map_plot

## All Parks

In [65]:
plot_mapping_data(mapping_data, title="all_parks")


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

## Disney Parks

In [95]:
plot_mapping_data(mapping_data, filter_column="Company", filter_value="Disney World", title="Disney_Parks")


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Animal Kingdom

In [85]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Animal Kingdom", scale=10, title="Animal_Kingdom")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Blizzard Beach

In [86]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Blizzard Beach", scale=15, title="Blizzard_Beach")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Epcot

In [87]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Epcot", scale=10, title="Epcot")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Hollywood Studios

In [88]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Hollywood Studios", 
                  scale=15, title="Hollywood_Studios")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Magic Kingdom

In [89]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Magic Kingdom", scale=15, title="Magic_Kingdom")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Typhoon Lagoon

In [90]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Typhoon Lagoon", 
                  scale=15, title="Typhoon_Lagoon")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

## Universal Parks

In [91]:
plot_mapping_data(mapping_data, filter_column="Company", filter_value="Universal", scale=15, title="Universal_Parks")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Islands of Adventure

In [92]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Islands of Adventure", 
                  scale=15, title="Islands_of_Adventure")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Universal Studios

In [93]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Universal Studios", 
                  scale=15, title="Universal_Studios")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)

### Volcano Bay

In [94]:
plot_mapping_data(mapping_data, filter_column="Theme_Park", filter_value="Volcano Bay", scale=15, title="Volcano_Bay")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (Ride_name,incident_count)